In [16]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import xgboost as xgb

loaded_model = xgb.XGBClassifier()
loaded_model.load_model('/content/drive/MyDrive/neural-accident-severity/models/best_xgb_model1.json')
print("مدل با موفقیت بارگذاری شد.")


مدل با موفقیت بارگذاری شد.


#ساخت  و ذخیره pipeline

In [9]:
print("ستون‌های ورودی sample_df:", sample_df.columns.tolist())
print("ستون‌های pipeline:", list(preprocessor.feature_names_in_))
print("تعداد ستون‌های ورودی:", sample_df.shape[1])
print("تعداد ستون‌های مدل انتظار دارد:", loaded_model.n_features_in_ if hasattr(loaded_model, 'n_features_in_') else 'نامشخص')


ستون‌های ورودی sample_df: ['mavane_did_id', 'naqayes_moaser_rah_id', 'vaze_hava_id', 'sharayet_sathe_rah_id', 'vaze_roshanaei_id', 'noe_barkhord_id', 'noe_shane_id', 'khatkeshi_id']
ستون‌های pipeline: ['mavane_did_id', 'naqayes_moaser_rah_id', 'vaze_hava_id', 'sharayet_sathe_rah_id', 'vaze_roshanaei_id', 'noe_barkhord_id', 'noe_shane_id', 'khatkeshi_id']
تعداد ستون‌های ورودی: 8
تعداد ستون‌های مدل انتظار دارد: 88


In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib

# --- تنظیمات ---
DATA_PATH = "/content/drive/MyDrive/neural-accident-severity/data/id_columns_step11.csv"
TARGET_COL = "noe_tasadof_id"
PREPROCESSOR_PATH = "/content/drive/MyDrive/neural-accident-severity/models/preprocessor.pkl"

# بارگذاری داده
df = pd.read_csv(DATA_PATH)
df = df.drop_duplicates()
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# پاکسازی مقادیر نامعتبر
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].replace(['nan', 'NaN', '', ' '], np.nan)
    df[col] = df[col].replace([np.inf, -np.inf], np.nan)

df = df[df[TARGET_COL].notna()]

X = df.drop(TARGET_COL, axis=1)
y = df[TARGET_COL]

# تقسیم داده
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# تعیین ستون‌ها
nominal_cols = [
    'khatkeshi_id', 'noe_barkhord_id', 'vaze_roshanaei_id',
    'sharayet_sathe_rah_id', 'noe_shane_id', 'vaze_hava_id',
    'mavane_did_id', 'naqayes_moaser_rah_id'
]
numeric_cols = [col for col in X.columns if col not in nominal_cols]

# ساخت pipeline پیش‌پردازش
nominal_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(transformers=[
    ('nom', nominal_transformer, nominal_cols),
    ('num', numeric_transformer, numeric_cols)
])

# آموزش pipeline روی داده آموزش
preprocessor.fit(X_train)

# ذخیره pipeline پیش‌پردازش
joblib.dump(preprocessor, PREPROCESSOR_PATH)
print(f"Pipeline پیش‌پردازش در {PREPROCESSOR_PATH} ذخیره شد.")

# اکنون می‌توانید مدل را جداگانه آموزش دهید:
# X_train_processed = preprocessor.transform(X_train)
# سپس مدل XGBoost را روی X_train_processed و y_train آموزش دهید.


Pipeline پیش‌پردازش در /content/drive/MyDrive/neural-accident-severity/models/preprocessor.pkl ذخیره شد.


In [15]:
import joblib
import xgboost as xgb
import pandas as pd

# بارگذاری pipeline و مدل
preprocessor = joblib.load('/content/drive/MyDrive/neural-accident-severity/models/preprocessor.pkl')
loaded_model = xgb.XGBClassifier()
loaded_model.load_model('/content/drive/MyDrive/neural-accident-severity/models/best_xgb_model1.json')

# دیکشنری‌های کامل تبدیل متن به عدد (همان‌طور که دارید تعریف کرده‌اید)
noe_barkhord_map = {
    'موتورسیکلت با وسیله نقلیه': 0,
    'وسیله نقلیه با یک وسیله نقلیه': 1,
    'وسیله نقلیه با عابر': 2,
    'موتورسیکلت با عابر': 3,
    'وسیله نقلیه با شیء ثابت': 4,
    'موتورسیکلت - واژگونی': 5,
    'وسیله نقلیه - واژگونی': 6,
    'موتورسیکلت با شیء ثابت': 7,
    'موتور با وسیله نقلیه پارک شده': 8,
    'وسیله نقلیه - خروج از جاده': 9,
    'سایر': 10,
    'وسیله نقلیه با دوچرخه': 11,
    'پرتاب سرنشین': 12,
    'موتورسیکلت با دوچرخه': 13,
    'نامشخص': 14,
    'موتورسیکلت - خروج از جاده': 15,
    'موتورسیکلت با موتورسیکلت': 16,
    'یک وسیله نقلیه': 17,
    'چند وسیله نقلیه': 18,
    'وسیله نقلیه با موتور سیکلت': 19,
    'دوچرخه با موتورسیکلت': 20,
    'مینی بوس به سواری': 21,
    'موتورسیکلت با چند وسیله دیگر': 22,
    'وسیله نقلیه پارک شده': 23,
    'وسیله نقلیه با چند وسیله نقلیه و عابر': 24,
    'موتورسیکلت': 25,
    'واژگونی و سقوط': 26,
    'وسیله نقلیه با موتورسیکلت': 27,
    'وسیله نقلیه با حیوان/احشام': 28,
    'واژگونی و سقوط، خروج از جاده': 29,
    'کامیون با وسیله نقلیه': 30,
    'وسیله نقلیه با چند وسیله نقلیه - عابر': 31,
    'وسیله نقلیه با دوچرخه و چند وسیله': 32,
    'موتورسیکلت با حیوان/احشام': 33,
    'چند برخوردی': 34
}

vaze_roshanaei_map = {
    'روز': 0,
    'شب با روشنایی کافی': 1,
    'غروب': 2,
    'شب بدون روشنایی کافی': 3,
    'نامشخص': 4,
    'طلوع': 5,
    'سایر': 6,
    'شب با روشنایی نامشخص': 7
}

sharayet_sathe_rah_map = {
    'خشک و معمولی': 0,
    'مرطوب و خیس': 1,
    'یخبندان و برفی': 2,
    'نامشخص': 3,
    'سایر': 4,
    'شنی و خاکی': 5,
    'گل آلود': 6
}

noe_shane_map = {
    'شانه ندارد': 0,
    'شانه خاکی': 1,
    'شانه آسفالته': 2,
    'نامشخص': 3
}

vaze_hava_map = {
    'صاف': 0,
    'ابری': 1,
    'بارانی': 2,
    'برفی': 3,
    'نامشخص': 4,
    'بارانی و ابری': 5,
    'مه آلود': 6,
    'غبارآلود': 7,
    'صاف - ابری': 8
}

khatkeshi_map = {
    'ندارد': 0,
    'منقطع': 1,
    'ممتد': 2,
    'دوبل': 3,
    'نامشخص': 4,
    'ممتد و دوبل': 5,
    'ممتد و منقطع': 6
}

def get_valid_input(prompt, valid_map):
    while True:
        val = input(prompt)
        if val in valid_map:
            return valid_map[val]
        else:
            print("ورودی نامعتبر. لطفاً یکی از موارد زیر را وارد کنید:")
            for k in valid_map.keys():
                print(f" - {k}")

# گرفتن ورودی کاربر
sample = {}
sample['mavane_did_id'] = 0
sample['naqayes_moaser_rah_id'] = 0
sample['vaze_hava_id'] = get_valid_input('وضعیت هوا: ', vaze_hava_map)
sample['sharayet_sathe_rah_id'] = get_valid_input('شرایط سطح راه: ', sharayet_sathe_rah_map)
sample['vaze_roshanaei_id'] = get_valid_input('وضعیت روشنایی: ', vaze_roshanaei_map)
sample['noe_barkhord_id'] = get_valid_input('نوع برخورد: ', noe_barkhord_map)
sample['noe_shane_id'] = get_valid_input('نوع شانه: ', noe_shane_map)
sample['khatkeshi_id'] = get_valid_input('خط‌کشی: ', khatkeshi_map)

sample_df = pd.DataFrame([sample])

# اعمال پیش‌پردازش
sample_processed = preprocessor.transform(sample_df)

print("تعداد ستون‌های خروجی pipeline:", sample_processed.shape[1])
print("تعداد ستون‌های مدل انتظار دارد:", loaded_model.n_features_in_)

# پیش‌بینی
y_pred = loaded_model.predict(sample_processed)

noe_tasadof_map = {0: 'جرحی', 1: 'خسارتی', 2: 'فوتی'}
print('پیش‌بینی مدل:', noe_tasadof_map.get(y_pred[0], 'نامشخص'))


وضعیت هوا: صاف
شرایط سطح راه: خشک و معمولی
وضعیت روشنایی: روز
نوع برخورد: موتورسیکلت
نوع شانه: شانه آسفالته
خط‌کشی: منقطع
تعداد ستون‌های خروجی pipeline: 88
تعداد ستون‌های مدل انتظار دارد: 88
پیش‌بینی مدل: جرحی
